In [1]:
# Load in libraries

import yfinance as yf

import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import Indicators_TS

import time
import psutil

In [2]:
# Inputs
initp = 50000*1.0 # total amount to risk (including factor for margin option)

test_time = '6mo'
weight_tf = 1 #in years (for volatility sharesize adjustment)

In [3]:
# Load in list of stock symbols from StatusLog spreadsheet

#format --> #Ticker StrategyName  TimeFrame Param1 Param2 Direction(long/short)

# Specify the path to your Excel file
file_path = 'StatusLog.xlsx'

# Read the Excel file
check_list = pd.read_excel(file_path, engine='openpyxl')

check_list 

,Ticker,Strategy,n_time,n_thresh,Direction
0,ABNB,Stoch_TF_Norm,14,0,1
1,ADSK,SMA_TF,100,0,1
2,ARKQ,SMA_TF,90,0,1
3,BA,SMA_TF,70,0,1
4,BAC,SMA_TF,90,0,1
5,BCO,SMA_TF,50,0,1
6,BURL,Stoch_TF_Norm,20,0,1
7,CAG,SMA_TF,105,0,1
8,CCL,Stoch_TF_Norm,16,0,1
9,CG,Stoch_TF_Norm,10,0,1


In [4]:
#  Define indicators and performance metrics

def getstockdata(stockname):
    tickerd = yf.Ticker(stockname) 
    data = tickerd.history(period = test_time, interval='1d', auto_adjust=False, actions=False) 
    data = data[['Open', 'High', 'Low', 'Close', 'Volume']]
    return data

def fetch_stock_data(symbolslist, start_date, end_date):
    # Create an empty DataFrame with the date range as the index
    dates = pd.date_range(start=start_date, end=end_date)
    data = pd.DataFrame(index=dates)
    
    # Add each stock's data as a new column
    for symbol in symbolslist:
        # Download data for each symbol
        stock_close = yf.download(symbol, start=start_date, end=end_date)['Close']
        
        # Create a unique column name if the symbol is duplicated
        col_name = symbol
        if col_name in data.columns:
            suffix = 2
            while f"{col_name}_{suffix}" in data.columns:
                suffix += 1
            col_name = f"{col_name}_{suffix}"
        
        # Join the stock data to the main DataFrame using `col_name`
        data[col_name] = stock_close
        
    data = data.dropna(axis = 0, how = 'any')
    return data

def check_active_status(data):
    if len(data) < 2:
        return "Not enough data"

    # Get the last and second-to-last values from the 'Active' column
    last_active = data['Active'].iloc[-1]
    second_last_active = data['Active'].iloc[-2]

    # Determine the status based on the last two values
    if last_active and second_last_active:
        return "on"  # Active remains on
    elif not last_active and not second_last_active:
        return "off"  # Active remains off
    elif last_active and not second_last_active:
        return "on NEW"  # Transitioned from off to on
    elif not last_active and second_last_active:
        return "off NEW"  # Transitioned from on to off
    else:
        return "error"


In [5]:
# Define strategy check functions
def SMA_TF_check(data, n1, n2):
    
    data['SMA1'] = Indicators_TS.SMA_TS(data, 1)
    data['SMA2'] = Indicators_TS.SMA_TS(data, n1)
    data['Active'] = False
    on_off = False

    data = data.dropna()

    for i in range(1, len(data)):
        if data['SMA1'].iloc[i] > data['SMA2'].iloc[i]:
            on_off = True
        elif data['SMA1'].iloc[i] < data['SMA2'].iloc[i]:
            on_off = False
        data.loc[data.index[i], 'Active'] = on_off
        
    status = check_active_status(data)
    return status

#============================================

def SMA_TF_SH_check(data, n1, n2):
   
    data['SMA1'] = Indicators_TS.SMA_TS(data, 1)
    data['SMA2'] = Indicators_TS.SMA_TS(data, n1)
    data['Active'] = False
    on_off = False

    data = data.dropna()    

    for i in range(1, len(data)):
        if data['SMA1'].iloc[i] < data['SMA2'].iloc[i] :
            on_off = True
        elif data['SMA1'].iloc[i] > data['SMA2'].iloc[i]:
            on_off = False
        data.loc[data.index[i], 'Active'] = on_off
        
    status = check_active_status(data)
    return status

#==================================

def RSI_TF_check(data,n1,n2):
    data['RSI'] = Indicators_TS.RSI_TS(data,n1)
    data['Active'] = False
    on_off = False

    for i in range(1, len(data)):
        if not on_off and data['RSI'].iloc[i] > n2:
            on_off = True
        elif on_off and data['RSI'].iloc[i] < n2-5:
            on_off = False
        #data['Active'].iloc[i] = on_off
        data.loc[data.index[i], 'Active'] = on_off
        
    status = check_active_status(data)
    return status

#==================================

def RSI_TF_SH_check(data,n1,n2):
    data['RSI'] = Indicators_TS.RSI_TS(data,n1)
    data['Active'] = False
    on_off = False

    for i in range(1, len(data)):
        if not on_off and data['RSI'].iloc[i] < n2:
            on_off = True
        elif on_off and data['RSI'].iloc[i] > n2+5:
            on_off = False
        #data['Active'].iloc[i] = on_off
        data.loc[data.index[i], 'Active'] = on_off
        
    status = check_active_status(data)
    return status

#==================================

def RSI_MR_check(data,n1,n2):
    data['RSI'] = Indicators_TS.RSI_TS(data,n1)
    data['Active'] = False
    on_off = False

    for i in range(1, len(data)):
        if not on_off and data['RSI'].iloc[i] < 35:
            on_off = True
        elif on_off and data['RSI'].iloc[i] > 65:
            on_off = False
        data.loc[data.index[i], 'Active'] = on_off
 
    status = check_active_status(data)
    return status

#==================================

def RSI_MR_SH_check(data,n1,n2):
    data['RSI'] = Indicators_TS.RSI_TS(data,n1)
    data['Active'] = False
    on_off = False

    for i in range(1, len(data)):
        if not on_off and data['RSI'].iloc[i] > 65:
            on_off = True
        elif on_off and data['RSI'].iloc[i] < 35:
            on_off = False
        data.loc[data.index[i], 'Active'] = on_off
 
    status = check_active_status(data)
    return status

#==================================

def Stoch_TF_Norm_check(data,n1,n2):
    data['Stoch'] = Indicators_TS.Stochastic_TS(data,n1,n2)
    data['Active'] = False
    on_off = False

    for i in range(1, len(data)):
        if not on_off and data['Stoch'].iloc[i] > 20 and data['Stoch'].iloc[i-1]<20:  
            on_off = True
        elif on_off and data['Stoch'].iloc[i] < 80 and data['Stoch'].iloc[i-1]>80:   
            on_off = False
        data.loc[data.index[i], 'Active'] = on_off
 
    status = check_active_status(data)
    return status

#==================================

def Stoch_MR_check(data,n1,n2):
    data['Stoch'] = Indicators_TS.Stochastic_TS(data,n1,n2)
    data['Active'] = False
    on_off = False

    for i in range(1, len(data)):
        if not on_off and data['Stoch'].iloc[i] < 20:  
            on_off = True
        elif on_off and data['Stoch'].iloc[i] > 80:  
            on_off = False
        data.loc[data.index[i], 'Active'] = on_off
 
    status = check_active_status(data)
    return status

#==================================

def Stoch_MR_SH_check(data,n1,n2):
    data['Stoch'] = Indicators_TS.Stochastic_TS(data,n1,n2)
    data['Active'] = False
    on_off = False

    for i in range(1, len(data)):
        if not on_off and data['Stoch'].iloc[i] > 80: 
            on_off = True
        elif on_off and data['Stoch'].iloc[i] < 20: 
            on_off = False
        data.loc[data.index[i], 'Active'] = on_off
 
    status = check_active_status(data)
    return status

#==================================

def Xtreme_Close_check(data,n1,n2):
    data['Stoch'] = Indicators_TS.Stochastic_TS(data,n1,n2)
    data['Stoch'] = round(data['Stoch'],2)
    
    data['Active'] = False
    on_off = False

    for i in range(1, len(data)):
        if not on_off and data['Stoch'].iloc[i] < 10:  
            on_off = True
        elif on_off and data['Stoch'].iloc[i] > 90:   
            on_off = False
        data.loc[data.index[i], 'Active'] = on_off
 
    status = check_active_status(data)
    return status

#==================================



In [6]:
#  Define different strategies

# moving average crossover
def SMA_TFFactory(n2): 
    class SMA_TF(Strategy):
        def init(self):
            close = self.data.Close
            self.sma1 = self.I(SMA, close, 1)
            self.sma2 = self.I(SMA, close, n2)

        def next(self):
            if crossover(self.sma1, self.sma2):
                self.buy()
            elif crossover(self.sma2, self.sma1):
                self.position.close()
                
    return SMA_TF
#=========================================================================================
# moving average crossover SHORT
def SMA_TF_SHFactory(n2): 
    class SMA_TF_SH(Strategy):
        def init(self):
            close = self.data.Close
            self.sma1 = self.I(SMA, close, 1)
            self.sma2 = self.I(SMA, close, n2)

        def next(self):
            if crossover(self.sma2, self.sma1):
                self.sell()
            elif crossover(self.sma1, self.sma2):
                self.position.close()
                
    return SMA_TF_SH
#=========================================================================================
# RSI meanrev
def RSI_MRFactory(n1):
    class RSI_MR(Strategy):
        def init(self):
            self.rsi = self.I(Indicators_TS.RSI_TS, data, n1)

        def next(self):
            if self.rsi < 35 and (not self.position or self.position.is_short):
                self.buy()
            elif self.rsi > 65 and self.position.is_long:
                self.position.close()

    return RSI_MR
#=========================================================================================
# RSI meanrev SHORT
def RSI_MR_SHFactory(n1):
    class RSI_MR_SH(Strategy):
        def init(self):
            self.rsi = self.I(Indicators_TS.RSI_TS, data, n1)

        def next(self):
            if self.rsi < 35 and self.position.is_short:
                self.position.close()
            elif self.rsi > 65 and (not self.position or self.position.is_long):
                self.sell() 

    return RSI_MR_SH
#=========================================================================================

# Stoch meanrev
def Stoch_MRFactory(n1):
    class Stoch_MR(Strategy):
        def init(self):
            data = self.data 
            self.stoch = self.I(Stochastic, data, n1)

        def next(self):
            if self.stoch < 20 and (not self.position or self.position.is_short):
                self.buy()
            elif self.stoch > 80 and self.position.is_long:
                self.position.close() #self.sell() 

    return Stoch_MR

#=========================================================================================

# Stoch meanrev SHORT
def Stoch_MR_SHFactory(n1):
    class Stoch_MR_SH(Strategy):
        def init(self):
            data = self.data
            self.stoch = self.I(Stochastic, data, n1)

        def next(self):
            if self.stoch < 20 and self.position.is_short:
                self.position.close()
            elif self.stoch > 80 and (not self.position or self.position.is_long):
                self.sell() 

    return Stoch_MR_SH
#=========================================================================================
# Stoch Norm
def Stoch_TF_NormFactory(n1):
    class Stoch_TF_Norm(Strategy):
        def init(self):
            data = self.data
            self.stoch = self.I(Stochastic, data, n1)
        def next(self):
            if crossover(self.stoch,20)and (not self.position or self.position.is_short):
                self.buy()
            elif crossover(80,self.stoch) and self.position.is_long:
                self.position.close()

    return Stoch_TF_Norm

#=========================================================================================
# Stoch Norm SHORT
def Stoch_TF_NormSHFactory(n1):
    class Stoch_TF_NormSH(Strategy):
        def init(self):
            data = self.data
            self.stoch = self.I(Stochastic, data, n1)

        def next(self):
            if crossover(self.stoch,20) and self.position.is_short:
                self.position.close()
            elif crossover(80,self.stoch) and (not self.position or self.position.is_long):
                self.sell()

    return Stoch_TF_NormSH
#=========================================================================================

# Xtreme_Close
def Xtreme_Close_Factory(n1):
    class Xtreme_Close(Strategy):

        def init(self):
            data = self.data
            self.stoch = self.I(Stochastic, data, 1)

        def next(self):
            if self.stoch <= 10 :
                self.position.close()
            elif self.stoch >= 90 :
                self.sell()

    return Xtreme_Close

In [7]:
# Check Strats ON/OFF

check_output = []
check_history = pd.DataFrame()

for index, row in check_list.iterrows():
    ticker = row['Ticker']
    stratn = row['Strategy']
    n1 = row['n_time']
    n2 = row['n_thresh']

    data = getstockdata(ticker)
    # make sure the data is up-to-date
    if data.index[-1].date() == datetime.today().date():
        datecheck = 'Updated'
    else:
        datecheck = 'OLD'
        
    stratf = stratn +'_check'
    checkfac = globals().get(stratf)

    
    try:
        check = checkfac(data,n1,n2)
    except TypeError as e:
        # Check if the error is specifically a 'NoneType' callable error
        if str(e) == "'NoneType' object is not callable":
            print(ticker," :Caught 'NoneType' error. Appending special row.")
            # Append a special row when the error occurs
            check = 'NA'
     
    check_output.append([ticker, stratn, n1, n2, check, datecheck])
    
# convert to df and sort
check_output=pd.DataFrame(check_output)
check_output.columns=['Ticker', 'Strategy', 'n1', 'n2', 'Status', 'Data Check']
check_output = check_output.sort_values(by='Ticker',ascending=False)
#print today's date and the latest date of the ticker dl to check for a match

# make the ticker name the index instead of a column
check_output.set_index('Ticker', inplace=True)
# check_output


In [8]:
# Get Stock Data

#Time
# Input the number of days
numyears = weight_tf
ttime = round(numyears*365,0)
# Get today's date
today = datetime.now()
# Calculate the date num_days ago
desired_date = today - timedelta(days=ttime)
# Format the date as a string
startdate = desired_date.strftime("%Y-%m-%d")
enddate = datetime.now().strftime("%Y-%m-%d")

stocksreduce = check_list['Ticker'].drop_duplicates().to_list()

data = fetch_stock_data(stocksreduce, startdate, enddate)


YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [9]:
# Create weightings based on Standard Deviation

# Get weightings
psetup =[]
#datastd = data
datastd = data.resample('W').last() #for switch to weekly for volatility calc
pstocks = data #=============
rstocks =  datastd.diff()/datastd.shift().abs()
std = rstocks.abs().mean()
i_std = 1 / std
weights = i_std/ i_std.sum()
print("Weightings check: Sum = ", weights.sum())

# for display
psetup = pd.DataFrame()
currentp = pstocks.iloc[-1]
currentp = np.abs(currentp)
stockallo= initp*weights/currentp
stocknum = np.floor(initp*weights/currentp)
ntot = (stocknum*currentp).sum()
psetup['Shares'] = stocknum
psetup['Intended #'] = stockallo.round(2)
psetup['Price'] = currentp.round(2)
psetup['Weights'] = weights.round(3)*100
psetup['Actual $'] = stocknum*currentp
psetup['% off'] = psetup['Actual $']/(initp*weights)*100
# psetup['% off'] = (psetup['Shares'])/psetup['Intended #']*100
# psetup['% off'] = (psetup['Shares']-psetup['Intended #'])/psetup['Intended #']*100
psetup['% off'] = psetup['% off'].round(3)
psetup = psetup.sort_values(by='% off',ascending=False)
psetup = psetup.sort_index()

Weightings check: Sum =  1.0


In [10]:
# Merge Dataframes and Display

# Merge Dataframes
psetupcol = psetup[['Shares','% off']] 
finallist = check_output.join(psetupcol, how="left")
finallist = finallist.sort_index()
finallist[['% off']] = finallist[['% off']].map("{:.1f}".format)
finallist.reset_index(inplace=True)

check_list_i =check_list.reset_index

# color coded
def color_text_status(val):
    if val == "on":
        return "color: green"  # Green for "Active"
    elif val == "off":
        return "color: red"    # Red for "Inactive"
    elif val == "on NEW":
        return "color: blue"   # Blue for a change in status
    elif val == "off NEW":
        return "color: blue"   # Blue for a change in status
    return ""  # Default (no style)
def color_text_poff(val):
    val = float(val)
    if val>=50 and val<60:
        return "color: orange"  

    return ""  # Default (no style)

# Apply styling and adjust shares to show neg numbers
styled_finallist = finallist.style.map(color_text_status, subset=["Status"]).map(color_text_poff, subset=["% off"])
finallist["Shares"] = finallist["Shares"].astype(int)
finallist.loc[finallist["Strategy"].str.endswith("SH"), "Shares"] = -finallist["Shares"]

# show additional data
print("# of strategies: ", check_list.shape[0])
print("Total $", psetup['Actual $'].sum().round().astype(int), "of allocated ", initp)
print("Today's Date", datetime.today().date())

# Display the styled DataFrame (works in Jupyter or notebook environments)
styled_finallist

# of strategies:  45
Total $ 46612 of allocated  50000.0
Today's Date 2025-08-10


,Ticker,Strategy,n1,n2,Status,Data Check,Shares,% off
0,ABNB,Stoch_TF_Norm,14,0,on,OLD,9,99.6
1,ADSK,SMA_TF,100,0,on,OLD,4,80.6
2,AMD,SMA_TF_SH,20,0,off,OLD,-4,80.9
3,ARKQ,SMA_TF,90,0,on,OLD,12,95.9
4,BA,SMA_TF,70,0,on,OLD,4,82.6
5,BAC,SMA_TF,90,0,on,OLD,32,97.6
6,BCO,SMA_TF,50,0,on,OLD,13,97.1
7,BURL,Stoch_TF_Norm,20,0,off,OLD,4,88.8
8,CAG,SMA_TF,105,0,off,OLD,93,99.6
9,CCL,Stoch_TF_Norm,16,0,off,OLD,29,98.6


In [11]:
# Prepare and Display currently owned list w/ stats

openlist = finallist[finallist["Status"] == "on"]
openlistf = openlist[['Ticker','Shares']]


#reformat for merge
currentp = pstocks.iloc[-1]
currentp = pd.DataFrame(currentp)
currentp.reset_index(inplace=True)
currentp.columns = ["Ticker", "Price"]
currentp

Clist = openlistf.merge(currentp[["Ticker", "Price"]], on="Ticker", how="left")

Clist["$ Amount"] = Clist["Shares"] * Clist["Price"].round(0)
Clist["Price"] = Clist["Price"].round(2)

# Report on Long/Short trades - how much of current portfolio is long or short

# Calculate the sum of the absolute values of "$ Amount"
total_absolute = Clist["$ Amount"].abs().sum().round(0)

# Calculate the sum of the positive and negative "$ Amount" separately
positive_sum = Clist.loc[Clist["$ Amount"] > 0, "$ Amount"].sum()
negative_sum = Clist.loc[Clist["$ Amount"] < 0, "$ Amount"].sum()

# Calculate the percentage of the total that is positive
percentage_positive = (positive_sum / total_absolute)*100
pinvest = total_absolute/initp*100;

# Print the outputs
print(f"Total $ Invested': {total_absolute}")
print(f"Percentage $ used of available: {pinvest:.2f}%")
print(f"Percentage of $ that is Long: {percentage_positive:.2f}%")

# Count the number of positive and negative rows
positive_count = (Clist["$ Amount"] > 0).sum()
negative_count = (Clist["$ Amount"] < 0).sum()

# Calculate the total number of rows
total_count = positive_count + negative_count

# Calculate the percentage of rows that are positive
percentage_positive_rows = (positive_count / total_count) * 100
print(f"Percentage of trades that are Long: {percentage_positive_rows:.2f}%")
Clist

Total $ Invested': 24724.0
Percentage $ used of available: 49.45%
Percentage of $ that is Long: 91.78%
Percentage of trades that are Long: 90.48%


,Ticker,Shares,Price,$ Amount
0,ABNB,9,121.02,1089.0
1,ADSK,4,290.68,1164.0
2,ARKQ,12,99.26,1188.0
3,BA,4,229.12,916.0
4,BAC,32,46.01,1472.0
5,BCO,13,105.49,1365.0
6,CMPS,121,4.39,484.0
7,CRSP,13,55.00,715.0
8,DE,3,510.37,1530.0
9,DIS,13,112.43,1456.0
